### Importing libraries

In [26]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

### Importing data

In [27]:
#importing the dataset
with open('intents.json') as intentsFile:
  intents = json.load(intentsFile)['intents']
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in intents:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({ "inputs":inputs,"tags":tags })
print(data)

                                     inputs       tags
0                                     hello   greeting
1                                  hi there   greeting
2                          nice to meet you   greeting
3                                        hi   greeting
4                                 hey there   greeting
5                                       hey   greeting
6                      hi, nice to meet you   greeting
7                               hello there   greeting
8                            anyone there ?   greeting
9                               knock knock   greeting
10                                      bye    goodbye
11                                  goodbye    goodbye
12                            see you later    goodbye
13                           im hopping off    goodbye
14                        talk to you later    goodbye
15                             i have to go    goodbye
16                                 whats up     howami
17        

### Pre-processing the data

In [28]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

### I/O length and Vocabulary

In [29]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

7
number of unique words :  65
output length:  7


### Neural network

In [30]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

model.save('model.h5')

Epoch 1/200
2/2 [==============================] - 2s 6ms/step - loss: 1.9458 - accuracy: 0.1800
Epoch 2/200
2/2 [==============================] - 0s 6ms/step - loss: 1.9429 - accuracy: 0.2400
Epoch 3/200
2/2 [==============================] - 0s 7ms/step - loss: 1.9405 - accuracy: 0.2600
Epoch 4/200
2/2 [==============================] - 0s 5ms/step - loss: 1.9383 - accuracy: 0.3200
Epoch 5/200
2/2 [==============================] - 0s 6ms/step - loss: 1.9357 - accuracy: 0.4600
Epoch 6/200
2/2 [==============================] - 0s 6ms/step - loss: 1.9334 - accuracy: 0.4800
Epoch 7/200
2/2 [==============================] - 0s 6ms/step - loss: 1.9308 - accuracy: 0.5000
Epoch 8/200
2/2 [==============================] - 0s 5ms/step - loss: 1.9281 - accuracy: 0.4800
Epoch 9/200
2/2 [==============================] - 0s 6ms/step - loss: 1.9252 - accuracy: 0.5200
Epoch 10/200
2/2 [==============================] - 0s 6ms/step - loss: 1.9224 - accuracy: 0.5000
Epoch 11/200
2/2 [===========